In [1]:
!pip install "stable-baselines3[extra]" --quiet
!pip install tqdm --quiet
%env MUJOCO_GL=enable
from myosuite.utils import gym
from showvid import show_video
import skvideo.io
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from stable_baselines3 import PPO


env: MUJOCO_GL=enable
pygame 2.6.1 (SDL 2.28.4, Python 3.8.20)
Hello from the pygame community. https://www.pygame.org/contribute.html
MyoSuite:> Registering Myo Envs


# MyoFingerReach example

In [ ]:
env = gym.make('myoFingerReachFixed-v0')

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=100000)
model.save("Reach_policy")

In [ ]:
# evaluate policy
all_rewards = []
for _ in tqdm(range(20)): # 20 random targets
  ep_rewards = []
  done = False
  obs, _ = env.reset()
  for _ in range(40):
      # get the next action from the policy
      action, _ = model.predict(obs)
      # take an action based on the current observation
      obs, reward, done, _, info = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 20 episodes")


In [ ]:
env.reset()
frames = []
obs, _ = env.reset()
for _ in tqdm(range(1000)):
    # Predict the best action based on the current observation
    action, _ = model.predict(obs)
    # Render the current frame from the environment
    frames.append(env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=-1))
    # Take the action and get the next observation
    obs, _, done, _, _ = env.step(action)
env.close()

In [ ]:
os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/Reach.mp4', np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})

# show in the notebook
show_video('videos/Reach.mp4')

# MyoDm example

In [2]:
import numpy as np
from myosuite.utils import gym
import os

env = gym.make('MyoHandTapMouseLift-v0')
env.reset()

from stable_baselines3 import PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1000)
# to train to convergence use more iterations e.g.
# model.learn(total_timesteps=1e7)
model.save("inspect_policy")

NameNotFound: Environment `MyoHandTapMouseLift` doesn't exist. Did you mean: `MyoHandMouseLift`?

In [ ]:
all_rewards = []
for _ in tqdm(range(20)): # 20 random targets
  ep_rewards = []
  done = False
  obs, _ = env.reset()
  for _ in range(40):
      # get the next action from the policy
      action, _ = model.predict(obs)
      # take an action based on the current observation
      obs, reward, done, _, info = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 20 episodes")


In [ ]:
env.reset()
# find geometries with ID == 1 which indicates the skins
geom_1_indices = np.where(env.sim.model.geom_group == 1)
# Change the alpha value to make it transparent
env.sim.model.geom_rgba[geom_1_indices, 3] = 0

env.sim.renderer.set_viewer_settings(
           render_actuator=True,
           render_tendon=True
       )


# find geometries with ID == 1 which indicates the skins
geom_1_indices = np.where(env.sim.model.geom_group == 1)
# Change the alpha value to make it transparent
env.sim.model.geom_rgba[geom_1_indices, 3] = 0

env.sim.renderer.set_viewer_settings(
           render_actuator=True,
           render_tendon=True
       )

pi = PPO.load("inspect_policy")
frames = []
for _ in range(400):
    frames.append(env.sim.renderer.render_offscreen(width=400, height=400, camera_id=3))
    o = env.get_obs()
    a = pi.predict(o)[0]
    next_o, r, done, *_, ifo = env.step(
        a
    )  # take an action based on the current observation
    
import skvideo.io
import os
os.makedirs("videos", exist_ok=True)
# make a local copy
skvideo.io.vwrite(
    "videos/inspect.mp4",

    np.asarray(frames),
    outputdict={"-pix_fmt": "yuv420p", "-r": "10"},
)




In [ ]:
show_video('videos/inspect.mp4')